In [ ]:
# Imported Libraries
import time, warnings, itertools, matplotlib.patches as mpatches, matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sns
from typing import Optional, Sequence, Dict, Tuple, List
from scipy.stats import norm
from collections import Counter
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

# Model Development Modules Import
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# SKLearn Modules Import
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection._split import _BaseKFold  # for type hinting
from sklearn.model_selection import (StratifiedKFold, GridSearchCV, ShuffleSplit, RandomizedSearchCV, 
    train_test_split, cross_val_score, learning_curve, cross_val_predict
)
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, f1_score, precision_score, average_precision_score,recall_score, roc_auc_score, 
    roc_curve, precision_recall_curve
)

warnings.filterwarnings("ignore")

# Import data
df=pd.read_csv("input/creditcard.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
# Good No Null Values!
df.isnull().sum().max()

In [ ]:
df.columns

In [ ]:
# The classes are heavily skewed we need to solve this issue later.
print("No Frauds", round(df["Class"].value_counts()[0] / len(df) * 100, 2), "% of the dataset")
print("Frauds", round(df["Class"].value_counts()[1] / len(df) * 100, 2), "% of the dataset")

In [ ]:
colors=["#0101DF", "#DF0101"]
sns.countplot(x="Class", data=df, palette=colors)
plt.title("Class Distributions \n (0: No Fraud || 1: Fraud)", fontsize=12)
fig, ax=plt.subplots(1, 2, figsize=(18, 4))

amount_val=df["Amount"].values
time_val=df["Time"].values

sns.histplot(amount_val, ax=ax[0], color="r")
ax[0].set_title("Distribution of Transaction Amount", fontsize=12)
ax[0].set_xlim([min(amount_val), max(amount_val)])

sns.histplot(time_val, ax=ax[1], color="b")
ax[1].set_title("Distribution of Transaction Time", fontsize=12)
ax[1].set_xlim([min(time_val), max(time_val)])

plt.show()

In [ ]:
# Since most of our data has already been scaled we should scale the columns that are left to scale (Amount and Time)

std_scaler=StandardScaler()
# RobustScaler is less prone to outliers.
rob_scaler=RobustScaler()

df["scaled_amount"]=rob_scaler.fit_transform(df["Amount"].values.reshape(-1, 1))
df["scaled_time"]=rob_scaler.fit_transform(df["Time"].values.reshape(-1, 1))
df.drop(["Time", "Amount"], axis=1, inplace=True)

scaled_amount=df["scaled_amount"]
scaled_time=df["scaled_time"]
df.drop(["scaled_amount", "scaled_time"], axis=1, inplace=True)
df.insert(0, "scaled_amount", scaled_amount)
df.insert(1, "scaled_time", scaled_time)
# Amount and Time are Scaled!
df.head()

In [ ]:
X=df.drop("Class", axis=1)
y=df["Class"]

sss=StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest=X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest=y.iloc[train_index], y.iloc[test_index]

# Convert to array
original_Xtrain=original_Xtrain.values
original_Xtest=original_Xtest.values
original_ytrain=original_ytrain.values
original_ytest=original_ytest.values

In [ ]:
# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest=train_test_split(X, y, test_size=0.2, random_state=42)

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label=np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label=np.unique(original_ytest, return_counts=True)
print("-" * 100)

print("Label Distributions: \n")
print(train_counts_label / len(original_ytrain))
print(test_counts_label / len(original_ytest))

# Since our classes are highly skewed we should make them equivalent in order to have 
# a normal distribution of the classes.

# Lets shuffle the data before creating the subsamples
df=df.sample(frac=1)
# amount of fraud classes 492 rows.
fraud_df=df.loc[df["Class"]==1]
non_fraud_df=df.loc[df["Class"]==0][:492]
normal_distributed_df=pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df=normal_distributed_df.sample(frac=1, random_state=42)
new_df.head()

In [ ]:
print("Distribution of the Classes in the subsample dataset")
print(new_df["Class"].value_counts() / len(new_df))

sns.countplot(x="Class", data=new_df, palette=colors)
plt.title("Equally Distributed Classes", fontsize=12)
plt.show()

In [ ]:
# Make sure we use the subsample in our correlation
f, (ax1, ax2)=plt.subplots(2, 1, figsize=(24, 20))

# Entire DataFrame
corr=df.corr()
sns.heatmap(corr, cmap="coolwarm_r", annot_kws={"size": 20}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix \n (don't use for reference)", fontsize=12)

sub_sample_corr=new_df.corr()
sns.heatmap(sub_sample_corr, cmap="coolwarm_r", annot_kws={"size": 20}, ax=ax2)
ax2.set_title("SubSample Correlation Matrix \n (use for reference)", fontsize=12)
plt.show()

In [ ]:
f, axes=plt.subplots(ncols=4, figsize=(20, 4))

# Negative Correlations with our Class (The lower our feature value the more likely it will be a fraud transaction)
sns.boxplot(x="Class", y="V17", data=new_df, palette=colors, ax=axes[0])
axes[0].set_title("V17 vs Class Negative Correlation")

sns.boxplot(x="Class", y="V14", data=new_df, palette=colors, ax=axes[1])
axes[1].set_title("V14 vs Class Negative Correlation")

sns.boxplot(x="Class", y="V12", data=new_df, palette=colors, ax=axes[2])
axes[2].set_title("V12 vs Class Negative Correlation")

sns.boxplot(x="Class", y="V10", data=new_df, palette=colors, ax=axes[3])
axes[3].set_title("V10 vs Class Negative Correlation")

plt.show()

In [ ]:
f, axes=plt.subplots(ncols=4, figsize=(20, 4))

# Positive correlations (The higher the feature the probability increases that it will be a fraud transaction)
sns.boxplot(x="Class", y="V11", data=new_df, palette=colors, ax=axes[0])
axes[0].set_title("V11 vs Class Positive Correlation")

sns.boxplot(x="Class", y="V4", data=new_df, palette=colors, ax=axes[1])
axes[1].set_title("V4 vs Class Positive Correlation")

sns.boxplot(x="Class", y="V2", data=new_df, palette=colors, ax=axes[2])
axes[2].set_title("V2 vs Class Positive Correlation")

sns.boxplot(x="Class", y="V19", data=new_df, palette=colors, ax=axes[3])
axes[3].set_title("V19 vs Class Positive Correlation")

plt.show()

In [ ]:
f, (ax1, ax2, ax3)=plt.subplots(1, 3, figsize=(20, 6))

# V14
v14_fraud_dist=new_df["V14"].loc[new_df["Class"]==1].values
sns.histplot(v14_fraud_dist, ax=ax1, kde=False, stat="density", color="#FB8861", bins=30)

mu, std=norm.fit(v14_fraud_dist)
xmin, xmax=ax1.get_xlim()
x=np.linspace(xmin, xmax, 100)
p=norm.pdf(x, mu, std)
ax1.plot(x, p, 'k', linewidth=2)
ax1.set_title("V14 Distribution\n(Fraud Transactions)", fontsize=12)

# V12
v12_fraud_dist=new_df["V12"].loc[new_df["Class"]==1].values
sns.histplot(v12_fraud_dist, ax=ax2, kde=False, stat="density", color="#56F9BB", bins=30)

mu, std=norm.fit(v12_fraud_dist)
xmin, xmax=ax2.get_xlim()
x=np.linspace(xmin, xmax, 100)
p=norm.pdf(x, mu, std)
ax2.plot(x, p, 'k', linewidth=2)
ax2.set_title("V12 Distribution\n(Fraud Transactions)", fontsize=12)

# V10
v10_fraud_dist=new_df["V10"].loc[new_df["Class"]==1].values
sns.histplot(v10_fraud_dist, ax=ax3, kde=False, stat="density", color="#C5B3F9", bins=30)

mu, std=norm.fit(v10_fraud_dist)
xmin, xmax=ax3.get_xlim()
x=np.linspace(xmin, xmax, 100)
p=norm.pdf(x, mu, std)
ax3.plot(x, p, 'k', linewidth=2)
ax3.set_title("V10 Distribution\n(Fraud Transactions)", fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# # -----> V14 Removing Outliers (Highest Negative Correlated with Labels)
v14_fraud=new_df["V14"].loc[new_df["Class"]==1].values
q25, q75=np.percentile(v14_fraud, 25), np.percentile(v14_fraud, 75)
print(f"Quartile 25: {q25} | Quartile 75: {q75}")
v14_iqr=q75 - q25
print(f"iqr: {v14_iqr}")

In [ ]:
v14_cut_off=v14_iqr * 1.5
v14_lower, v14_upper=q25 - v14_cut_off, q75 + v14_cut_off
print(f"Cut Off: {v14_cut_off}")
print(f"V14 Lower: {v14_lower}")
print(f"V14 Upper: {v14_upper}")

In [ ]:
outliers=[x for x in v14_fraud if x < v14_lower or x > v14_upper]
print(f"Feature V14 Outliers for Fraud Cases: {len(outliers)}")
print(f"V10 outliers:{outliers}")

In [ ]:
new_df=new_df.drop(new_df[(new_df["V14"] > v14_upper) | (new_df["V14"] < v14_lower)].index)
print("----" * 44)

In [ ]:
# -----> V12 removing outliers from fraud transactions
v12_fraud=new_df["V12"].loc[new_df["Class"]==1].values
q25, q75=np.percentile(v12_fraud, 25), np.percentile(v12_fraud, 75)

v12_iqr=q75 - q25
v12_cut_off=v12_iqr * 1.5
v12_lower, v12_upper=q25 - v12_cut_off, q75 + v12_cut_off
print(f"V12 Lower: {v12_lower}")
print(f"V12 Upper: {v12_upper}")

outliers=[x for x in v12_fraud if x < v12_lower or x > v12_upper]
print(f"V12 outliers: {outliers}")
print(f"Feature V12 Outliers for Fraud Cases: {len(outliers)}")

new_df=new_df.drop(new_df[(new_df["V12"] > v12_upper) | (new_df["V12"] < v12_lower)].index)
print(f"Number of Instances after outliers removal: {len(new_df)}")
print("----" * 44)

In [ ]:
# Removing outliers V10 Feature
v10_fraud=new_df["V10"].loc[new_df["Class"]==1].values
q25, q75=np.percentile(v10_fraud, 25), np.percentile(v10_fraud, 75)

v10_iqr=q75 - q25
v10_cut_off=v10_iqr * 1.5
v10_lower, v10_upper=q25 - v10_cut_off, q75 + v10_cut_off
print(f"V10 Lower: {v10_lower}")
print(f"V10 Upper: {v10_upper}")

outliers=[x for x in v10_fraud if x < v10_lower or x > v10_upper]
print(f"V10 outliers: {outliers}")
print(f"Feature V10 Outliers for Fraud Cases: {len(outliers)}")

new_df=new_df.drop(new_df[(new_df["V10"] > v10_upper) | (new_df["V10"] < v10_lower)].index)
print(f"Number of Instances after outliers removal: {len(new_df)}")

In [ ]:
f, (ax1, ax2, ax3)=plt.subplots(1, 3, figsize=(20, 6))

colors=["#B3F9C5", "#f9c5b3"]
# Boxplots with outliers removed
# Feature V14
sns.boxplot(x="Class", y="V14", data=new_df, ax=ax1, palette=colors)
ax1.set_title("V14 Feature Reduction of outliers", fontsize=12)
ax1.annotate("Fewer extreme outliers", xy=(0.98, -17.5), xytext=(0, -12), arrowprops=dict(facecolor="black"), fontsize=12)

# Feature 12
sns.boxplot(x="Class", y="V12", data=new_df, ax=ax2, palette=colors)
ax2.set_title("V12 Feature Reduction of outliers", fontsize=12)
ax2.annotate("Fewer extreme outliers", xy=(0.98, -17.3), xytext=(0, -12), arrowprops=dict(facecolor="black"), fontsize=12)

# Feature V10
sns.boxplot(x="Class", y="V10", data=new_df, ax=ax3, palette=colors)
ax3.set_title("V10 Feature Reduction of outliers", fontsize=12)
ax3.annotate("Fewer extreme outliers", xy=(0.95, -16.5), xytext=(0, -12), arrowprops=dict(facecolor="black"), fontsize=12)

plt.show()

In [ ]:
# New_df is from the random undersample data (fewer instances)
X=new_df.drop("Class", axis=1)
y=new_df["Class"]

In [ ]:
# T-SNE Implementation
t0=time.time()
X_reduced_tsne=TSNE(n_components=2, random_state=42).fit_transform(X.values)
t1=time.time()
print(f"T-SNE took {t1 - t0:.2f} s")

In [ ]:
# PCA Implementation
t0=time.time()
X_reduced_pca=PCA(n_components=2, random_state=42).fit_transform(X.values)
t1=time.time()
print(f"PCA took {t1 - t0:.2f} s")

In [ ]:
# TruncatedSVD
t0=time.time()
X_reduced_svd=TruncatedSVD(n_components=2, algorithm="randomized", random_state=42).fit_transform(X.values)
t1=time.time()
print(f"Truncated SVD took {t1 - t0:.2f} s")

In [ ]:
f, (ax1, ax2, ax3)=plt.subplots(1, 3, figsize=(24, 6))
# labels=['No Fraud', 'Fraud']
f.suptitle("Clusters using Dimensionality Reduction", fontsize=14)

blue_patch=mpatches.Patch(color="#0A0AFF", label="No Fraud")
red_patch=mpatches.Patch(color="#AF0000", label="Fraud")

# t-SNE scatter plot
ax1.scatter(X_reduced_tsne[:, 0], X_reduced_tsne[:, 1], c=(y==0), cmap="coolwarm", label="No Fraud", linewidths=2)
ax1.scatter(X_reduced_tsne[:, 0], X_reduced_tsne[:, 1], c=(y==1), cmap="coolwarm", label="Fraud", linewidths=2)
ax1.set_title("t-SNE", fontsize=14)
ax1.grid(True)
ax1.legend(handles=[blue_patch, red_patch])

# PCA scatter plot
ax2.scatter(X_reduced_pca[:, 0], X_reduced_pca[:, 1], c=(y==0), cmap="coolwarm", label="No Fraud", linewidths=2)
ax2.scatter(X_reduced_pca[:, 0], X_reduced_pca[:, 1], c=(y==1), cmap="coolwarm", label="Fraud", linewidths=2)
ax2.set_title("PCA", fontsize=14)
ax2.grid(True)
ax2.legend(handles=[blue_patch, red_patch])

# TruncatedSVD scatter plot
ax3.scatter(X_reduced_svd[:, 0], X_reduced_svd[:, 1], c=(y==0), cmap="coolwarm", label="No Fraud", linewidths=2)
ax3.scatter(X_reduced_svd[:, 0], X_reduced_svd[:, 1], c=(y==1), cmap="coolwarm", label="Fraud", linewidths=2)
ax3.set_title("Truncated SVD", fontsize=14)
ax3.grid(True)
ax3.legend(handles=[blue_patch, red_patch])

plt.show()

In [ ]:
# Undersampling before cross validating (prone to overfit)
X=new_df.drop("Class", axis=1)
y=new_df["Class"]

# Our data is already scaled we should split our training and test sets
# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

# Turn the values into an array for feeding the classification algorithms.
X_train=X_train.values
X_test=X_test.values
y_train=y_train.values
y_test=y_test.values

# Let's implement simple classifiers
classifiers={
    "LogisiticRegression": LogisticRegression(), 
    "KNearest": KNeighborsClassifier(), 
    "Support Vector Classifier": SVC(), 
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

# Wow our scores are getting even high scores even when applying cross validation.
for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score=cross_val_score(classifier, X_train, y_train, cv=5)
    print(f"Classifiers: {classifier.__class__.__name__} has a training score of {round(training_score.mean(), 2) * 100} % accuracy score")

In [ ]:
# Use GridSearchCV to find the best parameters.

# Logistic Regression
log_reg_params={"penalty": ["l1", "l2"], "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_log_reg=GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)
# LogisticRegression best estimator
log_reg=grid_log_reg.best_estimator_

# KNN
knears_params={"n_neighbors": list(range(2, 5, 1)), "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]}
grid_knears=GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(X_train, y_train)
# KNears best estimator
knears_neighbors=grid_knears.best_estimator_

# Support Vector Classifier
svc_params={"C": [0.5, 0.7, 0.9, 1], "kernel": ["rbf", "poly", "sigmoid", "linear"]}
grid_svc=GridSearchCV(SVC(), svc_params)
grid_svc.fit(X_train, y_train)
# SVC best estimator
svc=grid_svc.best_estimator_

# DecisionTree Classifier
tree_params={"criterion": ["gini", "entropy"], "max_depth": list(range(2, 4, 1)), "min_samples_leaf": list(range(5, 7, 1))}
grid_tree=GridSearchCV(DecisionTreeClassifier(), tree_params)
grid_tree.fit(X_train, y_train)
# DecisionTree best estimator
tree_clf=grid_tree.best_estimator_

In [ ]:
# Overfitting Case
log_reg_score=cross_val_score(log_reg, X_train, y_train, cv=5)
print(f"Logistic Regression Cross Validation Score: {round(log_reg_score.mean() * 100, 2).astype(str)}%")

knears_score=cross_val_score(knears_neighbors, X_train, y_train, cv=5)
print(f"Knears Neighbors Cross Validation Score {round(knears_score.mean() * 100, 2).astype(str)} %")

svc_score=cross_val_score(svc, X_train, y_train, cv=5)
print(f"Support Vector Classifier Cross Validation Score{round(svc_score.mean() * 100, 2).astype(str)} %")

tree_score=cross_val_score(tree_clf, X_train, y_train, cv=5)
print(f"DecisionTree Classifier Cross Validation Score {round(tree_score.mean() * 100, 2).astype(str)} %")

In [ ]:
# We will undersample during cross validating
undersample_X=df.drop("Class", axis=1)
undersample_y=df["Class"]

for train_index, test_index in sss.split(undersample_X, undersample_y):
    print("Train:", train_index, "Test:", test_index)
    undersample_Xtrain, undersample_Xtest=(undersample_X.iloc[train_index], undersample_X.iloc[test_index])
    undersample_ytrain, undersample_ytest=(undersample_y.iloc[train_index], undersample_y.iloc[test_index])

undersample_Xtrain=undersample_Xtrain.values
undersample_Xtest=undersample_Xtest.values
undersample_ytrain=undersample_ytrain.values
undersample_ytest=undersample_ytest.values


# Implementing NearMiss Technique
# Distribution of NearMiss (Just to see how it distributes the labels we won't use these variables)
X_nearmiss, y_nearmiss=NearMiss().fit_resample(undersample_X.values, undersample_y.values)
print(f"NearMiss Label Distribution: {Counter(y_nearmiss)}")

# Cross Validating the right way
undersample_accuracy=[]
undersample_precision=[]
undersample_recall=[]
undersample_f1=[]
undersample_auc=[]

for train, test in sss.split(undersample_Xtrain, undersample_ytrain):
    undersample_pipeline=imbalanced_make_pipeline(NearMiss(sampling_strategy="majority"), log_reg)  # SMOTE happens during Cross Validation not before..
    undersample_model=undersample_pipeline.fit(undersample_Xtrain[train], undersample_ytrain[train])
    undersample_prediction=undersample_model.predict(undersample_Xtrain[test])

    undersample_accuracy.append(undersample_pipeline.score(original_Xtrain[test], original_ytrain[test]))
    undersample_precision.append(precision_score(original_ytrain[test], undersample_prediction))
    undersample_recall.append(recall_score(original_ytrain[test], undersample_prediction))
    undersample_f1.append(f1_score(original_ytrain[test], undersample_prediction))
    undersample_auc.append(roc_auc_score(original_ytrain[test], undersample_prediction))

In [ ]:
# Let's Plot LogisticRegression Learning Curve
# def plot_learning_curve(estimator1, estimator2, estimator3, estimator4, X, y, ylim=None, cv=None, n_jobs=1, train_sizes=np.linspace(0.1, 1.0, 5)):
#     f, ((ax1, ax2), (ax3, ax4))=plt.subplots(2, 2, figsize=(20, 14), sharey=True)
    
#     if ylim is not None:
#         plt.ylim(*ylim)
    
#     # First Estimator
#     train_sizes, train_scores, test_scores=learning_curve(estimator1, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean=np.mean(train_scores, axis=1)
#     train_scores_std=np.std(train_scores, axis=1)
#     test_scores_mean=np.mean(test_scores, axis=1)
#     test_scores_std=np.std(test_scores, axis=1)
#     ax1.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="#ff9124")
#     ax1.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
#     ax1.plot(train_sizes, train_scores_mean, "o-", color="#ff9124", label="Training score")
#     ax1.plot(train_sizes, test_scores_mean, "o-", color="#2492ff", label="Cross-validation score")
#     ax1.set_title("Logistic Regression Learning Curve", fontsize=12)
#     ax1.set_xlabel("Training size (m)")
#     ax1.set_ylabel("Score")
#     ax1.grid(True)
#     ax1.legend(loc="best")

#     # Second Estimator
#     train_sizes, train_scores, test_scores=learning_curve(estimator2, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean=np.mean(train_scores, axis=1)
#     train_scores_std=np.std(train_scores, axis=1)
#     test_scores_mean=np.mean(test_scores, axis=1)
#     test_scores_std=np.std(test_scores, axis=1)
#     ax2.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="#ff9124")
#     ax2.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
#     ax2.plot(train_sizes, train_scores_mean, "o-", color="#ff9124", label="Training score")
#     ax2.plot(train_sizes, test_scores_mean, "o-", color="#2492ff", label="Cross-validation score")
#     ax2.set_title("Knears Neighbors Learning Curve", fontsize=12)
#     ax2.set_xlabel("Training size (m)")
#     ax2.set_ylabel("Score")
#     ax2.grid(True)
#     ax2.legend(loc="best")

#     # Third Estimator
#     train_sizes, train_scores, test_scores=learning_curve(estimator3, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean=np.mean(train_scores, axis=1)
#     train_scores_std=np.std(train_scores, axis=1)
#     test_scores_mean=np.mean(test_scores, axis=1)
#     test_scores_std=np.std(test_scores, axis=1)
#     ax3.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="#ff9124")
#     ax3.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
#     ax3.plot(train_sizes, train_scores_mean, "o-", color="#ff9124", label="Training score")
#     ax3.plot(train_sizes, test_scores_mean, "o-", color="#2492ff", label="Cross-validation score")
#     ax3.set_title("Support Vector Classifier \n Learning Curve", fontsize=12)
#     ax3.set_xlabel("Training size (m)")
#     ax3.set_ylabel("Score")
#     ax3.grid(True)
#     ax3.legend(loc="best")

#     # Fourth Estimator
#     train_sizes, train_scores, test_scores=learning_curve(estimator4, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean=np.mean(train_scores, axis=1)
#     train_scores_std=np.std(train_scores, axis=1)
#     test_scores_mean=np.mean(test_scores, axis=1)
#     test_scores_std=np.std(test_scores, axis=1)
#     ax4.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="#ff9124")
#     ax4.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
#     ax4.plot(train_sizes, train_scores_mean, "o-", color="#ff9124", label="Training score")
#     ax4.plot( train_sizes, test_scores_mean, "o-", color="#2492ff", label="Cross-validation score")
#     ax4.set_title("Decision Tree Classifier \n Learning Curve", fontsize=12)
#     ax4.set_xlabel("Training size (m)")
#     ax4.set_ylabel("Score")
#     ax4.grid(True)
#     ax4.legend(loc="best")
#     return plt

# cv=ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
# plot_learning_curve(log_reg, knears_neighbors, svc, tree_clf, X_train, y_train, (0.87, 1.01), cv=cv, n_jobs=4)

#--------------------------------------------------------------------------------------------------------------------------------------------------


def _plot_estimator_curve(ax: plt.Axes, estimator, X, y, cv: _BaseKFold, n_jobs: int, train_sizes: np.ndarray, title: str, ylim: Optional[Sequence[float]]=None) -> None:
    """
    Helper to compute and plot a single estimator's learning curve on the given axis.

    Args:
        ax: Matplotlib axis to draw on.
        estimator: A scikit-learn estimator with fit/predict methods.
        X: Feature matrix.
        y: Target vector.
        cv: Cross-validation splitter instance.
        n_jobs: Number of jobs for parallel processing.
        train_sizes: Relative or absolute sizes of training sets.
        title: Title for this subplot.
        ylim: Optional tuple (ymin, ymax) to set y-axis limits.
    """
    # Compute learning curve metrics
    train_sizes_abs, train_scores, test_scores=learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)

    # Calculate mean and std for training and validation scores
    train_mean=np.mean(train_scores, axis=1)
    train_std=np.std(train_scores, axis=1)
    test_mean=np.mean(test_scores, axis=1)
    test_std=np.std(test_scores, axis=1)

    # Plot shading for std deviation
    ax.fill_between(train_sizes_abs, train_mean - train_std, train_mean + train_std, alpha=0.1, color="#ff9124")
    ax.fill_between(train_sizes_abs, test_mean - test_std, test_mean + test_std, alpha=0.1, color="#2492ff")
    # Plot mean lines
    ax.plot(train_sizes_abs, train_mean, marker="o", linestyle="-", color="#ff9124", label="Training score")
    ax.plot(train_sizes_abs, test_mean, marker="o", linestyle="-", color="#2492ff", label="Cross-validation score")

    # Set axis labels, title, grid, and legend
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Training size (m)")
    ax.set_ylabel("Score")
    if ylim is not None:
        ax.set_ylim(*ylim)
    ax.grid(True)
    ax.legend(loc="best")


def plot_learning_curves(estimators: Sequence, titles: Sequence[str], X, y, ylim: Optional[Sequence[float]]=None, cv: Optional[_BaseKFold]=None, n_jobs: int=1, train_sizes: Optional[np.ndarray]=None) -> plt.Figure:
    """
    Plot learning curves for multiple estimators in a 2x2 grid.

    Args:
        estimators: Sequence of four scikit-learn estimators.
        titles: Sequence of four titles corresponding to each estimator.
        X: Feature matrix.
        y: Target vector.
        ylim: Optional (ymin, ymax) for all plots.
        cv: Cross-validation splitter. If None, defaults to None in sklearn.
        n_jobs: Number of parallel jobs.
        train_sizes: Array of training set sizes to evaluate; defaults to np.linspace(0.1, 1.0, 5).

    Returns:
        The matplotlib Figure object containing the plots.
    """
    # Default train sizes from 10% to 100% of data
    if train_sizes is None:
        train_sizes=np.linspace(0.1, 1.0, 5)

    # Create a 2x2 subplot grid
    fig, axes=plt.subplots(2, 2, figsize=(20, 14), sharey=True)
    axes_flat=axes.ravel()

    # Plot each estimator
    for ax, estimator, title in zip(axes_flat, estimators, titles):
        _plot_estimator_curve(ax=ax, estimator=estimator, X=X, y=y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, title=title, ylim=ylim)

    fig.tight_layout()
    return fig

# Example usage:
cv=ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
estimators=[log_reg, knears_neighbors, svc, tree_clf]
titles=["Logistic Regression Learning Curve", "K-Neighbors Learning Curve", "Support Vector Classifier Learning Curve", "Decision Tree Classifier Learning Curve"]

fig=plot_learning_curves(estimators, titles, X_train, y_train, ylim=(0.87, 1.01), cv=cv, n_jobs=4)
plt.show()

In [ ]:
# Create a DataFrame with all the scores and the classifiers names.
# log_reg_pred=cross_val_predict(log_reg, X_train, y_train, cv=5, method="decision_function")
# knears_pred=cross_val_predict(knears_neighbors, X_train, y_train, cv=5)
# svc_pred=cross_val_predict(svc, X_train, y_train, cv=5, method="decision_function")
# tree_pred=cross_val_predict(tree_clf, X_train, y_train, cv=5)

# print("Logistic Regression: ", roc_auc_score(y_train, log_reg_pred))
# print("KNears Neighbors: ", roc_auc_score(y_train, knears_pred))
# print("Support Vector Classifier: ", roc_auc_score(y_train, svc_pred))
# print("Decision Tree Classifier: ", roc_auc_score(y_train, tree_pred))

# log_fpr, log_tpr, log_thresold=roc_curve(y_train, log_reg_pred)
# knear_fpr, knear_tpr, knear_threshold=roc_curve(y_train, knears_pred)
# svc_fpr, svc_tpr, svc_threshold=roc_curve(y_train, svc_pred)
# tree_fpr, tree_tpr, tree_threshold=roc_curve(y_train, tree_pred)


# def graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr):
#     plt.figure(figsize=(16, 8))
#     plt.title("ROC Curve \n Top 4 Classifiers", fontsize=12)
#     plt.plot(log_fpr, log_tpr, label=f"Logistic Regression Classifier Score: {roc_auc_score(y_train, log_reg_pred):.4f}")
#     plt.plot(knear_fpr, knear_tpr, label=f"KNears Neighbors Classifier Score: {roc_auc_score(y_train, knears_pred):.4f}")
#     plt.plot(svc_fpr, svc_tpr, label=f"Support Vector Classifier Score: {roc_auc_score(y_train, svc_pred):.4f}")
#     plt.plot(tree_fpr, tree_tpr, label=f"Decision Tree Classifier Score: {roc_auc_score(y_train, tree_pred):.4f}")
#     plt.plot([0, 1], [0, 1], "k--")
#     plt.axis([-0.01, 1, 0, 1])
#     plt.xlabel("False Positive Rate", fontsize=12)
#     plt.ylabel("True Positive Rate", fontsize=12)
#     plt.annotate("Minimum ROC Score of 50% \n (This is the minimum score to get)", xy=(0.5, 0.5), xytext=(0.6, 0.3), arrowprops=dict(facecolor="#6E726D", shrink=0.05))
#     plt.legend()

# graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr)
# plt.show()

# def logistic_roc_curve(log_fpr, log_tpr):
#     plt.figure(figsize=(12, 8))
#     plt.title("Logistic Regression ROC Curve", fontsize=12)
#     plt.plot(log_fpr, log_tpr, "b-", linewidth=2)
#     plt.plot([0, 1], [0, 1], "r--")
#     plt.xlabel("False Positive Rate", fontsize=12)
#     plt.ylabel("True Positive Rate", fontsize=12)
#     plt.axis([-0.01, 1, 0, 1])

# logistic_roc_curve(log_fpr, log_tpr)
# plt.show()

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def compute_cross_val_predictions(classifiers: Dict[str, object], X, y, cv: int = 5, use_decision_fn: List[str] = None) -> Dict[str, np.ndarray]:
    """
    Compute cross-validated predictions or decision function scores for each classifier.

    Args:
        classifiers: Mapping of classifier names to instantiated models.
        X: Feature matrix.
        y: Target vector.
        cv: Number of folds for cross-validation.
        use_decision_fn: List of classifier names to use decision_function instead of predict.

    Returns:
        Dictionary mapping classifier names to prediction score arrays.
    """
    preds = {}
    for name, clf in classifiers.items():
        # Choose method: decision_function if available/requested, else default predict
        if use_decision_fn and name in use_decision_fn and hasattr(clf, 'decision_function'):
            preds[name] = cross_val_predict(clf, X, y, cv=cv, method='decision_function')
        else:
            preds[name] = cross_val_predict(clf, X, y, cv=cv)
    return preds


def compute_scores_and_curves(preds: Dict[str, np.ndarray], y) -> Tuple[pd.DataFrame, Dict[str, Tuple[np.ndarray, np.ndarray, np.ndarray]]]:
    """
    Calculate ROC AUC scores and ROC curve points for each classifier.

    Args:
        preds: Mapping of classifier names to predicted scores or labels.
        y: True target vector.

    Returns:
        scores_df: DataFrame with columns ['classifier', 'roc_auc'].
        roc_data: Mapping of classifier names to (fpr, tpr, thresholds).
    """
    records = []
    roc_data = {}
    for name, scores in preds.items():
        # Compute AUC
        records.append({'classifier': name, 'roc_auc': roc_auc_score(y, scores)})

        # Compute ROC curve
        fpr, tpr, thresh = roc_curve(y, scores)
        roc_data[name] = (fpr, tpr, thresh)

    scores_df = pd.DataFrame.from_records(records)
    return scores_df, roc_data


def plot_multiple_roc(roc_data: Dict[str, Tuple[np.ndarray, np.ndarray, np.ndarray]], scores_df: pd.DataFrame, figsize: Tuple[int, int] = (16, 8), title: str = 'ROC Curve - Multiple Classifiers') -> plt.Figure:
    """
    Plot ROC curves for multiple classifiers on one figure.

    Args:
        roc_data: Mapping of classifier names to ROC curve arrays.
        scores_df: DataFrame with 'classifier' and 'roc_auc' columns.
        figsize: Figure size tuple.
        title: Plot title.

    Returns:
        Matplotlib Figure object.
    """
    plt.figure(figsize=figsize)
    plt.title(title, fontsize=14)

    # Plot each ROC curve with AUC in label
    for _, row in scores_df.iterrows():
        name = row['classifier']
        auc = row['roc_auc']
        fpr, tpr, _ = roc_data[name]
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.4f})")

    # Diagonal chance line
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.grid(True)
    plt.legend(loc='lower right')
    return plt.gcf()


def plot_single_roc(roc_data: Tuple[np.ndarray, np.ndarray, np.ndarray], title: str = 'ROC Curve', figsize: Tuple[int, int] = (12, 8)) -> plt.Figure:
    """
    Plot a single ROC curve.

    Args:
        roc_data: Tuple of (fpr, tpr, thresholds).
        title: Plot title.
        figsize: Figure size tuple.

    Returns:
        Matplotlib Figure object.
    """
    fpr, tpr, _ = roc_data
    plt.figure(figsize=figsize)
    plt.title(title, fontsize=14)
    plt.plot(fpr, tpr, linestyle='-', lw=2)
    plt.plot([0, 1], [0, 1], 'r--', lw=1)
    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.grid(True)
    return plt.gcf()


classifiers = {'Logistic Regression': log_reg, 'KNeighbors': knears_neighbors, 'SVC': svc, 'Decision Tree': tree_clf}
preds = compute_cross_val_predictions(classifiers, X_train, y_train, cv=5, use_decision_fn=['Logistic Regression', 'SVC'])
scores_df, roc_data = compute_scores_and_curves(preds, y_train)
print(scores_df)
fig_multi = plot_multiple_roc(roc_data, scores_df)
fig_single = plot_single_roc(roc_data['Logistic Regression'], 'Logistic Regression ROC')
plt.show()

In [ ]:
precision, recall, threshold=precision_recall_curve(y_train, preds["log_reg_pred"])
y_pred=log_reg.predict(X_train)

# Overfitting Case
print("---" * 45)
print("Overfitting: \n")
print(f"Recall Score: {recall_score(y_train, y_pred):.2f}")
print(f"Precision Score: {precision_score(y_train, y_pred):.2f}")
print(f"F1 Score: {f1_score(y_train, y_pred):.2f}")
print(f"Accuracy Score: {accuracy_score(y_train, y_pred):.2f}")
print("---" * 45)

# How it should look like
print("---" * 45)
print("How it should be: \n")
print(f"Accuracy Score: {np.mean(undersample_accuracy):.2f}")
print(f"Precision Score: {np.mean(undersample_precision):.2f}")
print(f"Recall Score: {np.mean(undersample_recall):.2f}")
print(f"F1 Score: {np.mean(undersample_f1):.2f}")
print("---" * 45)

In [ ]:
undersample_y_score=log_reg.decision_function(original_Xtest)
undersample_average_precision=average_precision_score(original_ytest, undersample_y_score)
print(f"Average precision-recall score: {undersample_average_precision:.2f}")

In [ ]:
fig=plt.figure(figsize=(12, 6))
precision, recall, _=precision_recall_curve(original_ytest, undersample_y_score)

plt.step(recall, precision, color="#004a93", alpha=0.2, where="post")
plt.fill_between(recall, precision, step="post", alpha=0.2, color="#48a6ff")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(f"UnderSampling Precision-Recall curve:\nAverage Precision-Recall Score={undersample_average_precision:0.2f}", fontsize=12)

In [ ]:
print(f"Length of X_train: {len(original_Xtrain)} | Length of y_train: {len(original_ytrain)}")
print(f"Length of X_test: {len(original_Xtest)} | Length of y_test: {len(original_ytest)}")

# List to append the score and then find the average
accuracy_lst=[]
precision_lst=[]
recall_lst=[]
f1_lst=[]
auc_lst=[]

# Classifier with optimal parameters
# log_reg_sm=grid_log_reg.best_estimator_
log_reg_sm=LogisticRegression()
rand_log_reg=RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)

# Implementing SMOTE Technique
# Cross Validating the right way
# Parameters
log_reg_params={"penalty": ["l1", "l2"], "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
for train, test in sss.split(original_Xtrain, original_ytrain):
    pipeline=imbalanced_make_pipeline(SMOTE(sampling_strategy="minority"), rand_log_reg)  # SMOTE happens during Cross Validation not before..
    model=pipeline.fit(original_Xtrain[train], original_ytrain[train])
    best_est=rand_log_reg.best_estimator_
    prediction=best_est.predict(original_Xtrain[test])

    accuracy_lst.append(pipeline.score(original_Xtrain[test], original_ytrain[test]))
    precision_lst.append(precision_score(original_ytrain[test], prediction))
    recall_lst.append(recall_score(original_ytrain[test], prediction))
    f1_lst.append(f1_score(original_ytrain[test], prediction))
    auc_lst.append(roc_auc_score(original_ytrain[test], prediction))

print("---" * 45)
print(f"\naccuracy: {np.mean(accuracy_lst)} \nprecision: {np.mean(precision_lst)} \nrecall: {np.mean(recall_lst)} \nf1: {np.mean(f1_lst)}")
print("---" * 45)

In [ ]:
labels=["No Fraud", "Fraud"]
smote_prediction=best_est.predict(original_Xtest)
print(classification_report(original_ytest, smote_prediction, target_names=labels))

y_score=best_est.decision_function(original_Xtest)
average_precision=average_precision_score(original_ytest, y_score)
print(f"Average precision-recall score: {average_precision:0.2f}")

In [ ]:
fig=plt.figure(figsize=(12, 6))
precision, recall, _=precision_recall_curve(original_ytest, y_score)

plt.step(recall, precision, color="r", alpha=0.2, where="post")
plt.fill_between(recall, precision, step="post", alpha=0.2, color="#F59B00")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(f"OverSampling Precision-Recall curve: \nAverage Precision-Recall Score ={average_precision:0.2f}", fontsize=12)

In [ ]:
# SMOTE Technique (OverSampling) After splitting and Cross Validating
sm=SMOTE(sampling_strategy="minority", random_state=42)

# This will be the data were we are going to
Xsm_train, ysm_train=sm.fit_resample(original_Xtrain, original_ytrain)

# We Improve the score by 2% points approximately
# Implement GridSearchCV and the other models.

# Logistic Regression
t0=time.time()
log_reg_sm=grid_log_reg.best_estimator_
log_reg_sm.fit(Xsm_train, ysm_train)
t1=time.time()
print(f"Fitting oversample data took :{t1 - t0} sec")

In [ ]:
# Logistic Regression fitted using SMOTE technique
y_pred_log_reg=log_reg_sm.predict(X_test)

# Other models fitted with UnderSampling
y_pred_knear=knears_neighbors.predict(X_test)
y_pred_svc=svc.predict(X_test)
y_pred_tree=tree_clf.predict(X_test)

log_reg_cf=confusion_matrix(y_test, y_pred_log_reg)
kneighbors_cf=confusion_matrix(y_test, y_pred_knear)
svc_cf=confusion_matrix(y_test, y_pred_svc)
tree_cf=confusion_matrix(y_test, y_pred_tree)

In [ ]:
fig, ax=plt.subplots(2, 2, figsize=(22, 12))

sns.heatmap(log_reg_cf, ax=ax[0][0], annot=True, cmap=plt.cm.copper)
ax[0, 0].set_title("Logistic Regression \n Confusion Matrix", fontsize=12)
ax[0, 0].set_xticklabels(["", ""], fontsize=12, rotation=90)
ax[0, 0].set_yticklabels(["", ""], fontsize=12, rotation=360)

sns.heatmap(kneighbors_cf, ax=ax[0][1], annot=True, cmap=plt.cm.copper)
ax[0][1].set_title("KNearsNeighbors \n Confusion Matrix", fontsize=12)
ax[0][1].set_xticklabels(["", ""], fontsize=12, rotation=90)
ax[0][1].set_yticklabels(["", ""], fontsize=12, rotation=360)

sns.heatmap(svc_cf, ax=ax[1][0], annot=True, cmap=plt.cm.copper)
ax[1][0].set_title("Suppor Vector Classifier \n Confusion Matrix", fontsize=12)
ax[1][0].set_xticklabels(["", ""], fontsize=12, rotation=90)
ax[1][0].set_yticklabels(["", ""], fontsize=12, rotation=360)

sns.heatmap(tree_cf, ax=ax[1][1], annot=True, cmap=plt.cm.copper)
ax[1][1].set_title("DecisionTree Classifier \n Confusion Matrix", fontsize=12)
ax[1][1].set_xticklabels(["", ""], fontsize=12, rotation=90)
ax[1][1].set_yticklabels(["", ""], fontsize=12, rotation=360)

plt.show()

In [ ]:
print("Logistic Regression:", classification_report(y_test, y_pred_log_reg))
print("KNears Neighbors:", classification_report(y_test, y_pred_knear))
print("Support Vector Classifier:", classification_report(y_test, y_pred_svc))
print("Support Vector Classifier:", classification_report(y_test, y_pred_tree))

In [ ]:
# Final Score in the test set of logistic regression
# Logistic Regression with Under-Sampling
y_pred=log_reg.predict(X_test)
undersample_score=accuracy_score(y_test, y_pred)

# Logistic Regression with SMOTE Technique (Better accuracy with SMOTE t)
y_pred_sm=best_est.predict(original_Xtest)
oversample_score=accuracy_score(original_ytest, y_pred_sm)

In [ ]:
d={"Technique": ["Random UnderSampling", "Oversampling (SMOTE)"], "Score": [undersample_score, oversample_score]}
final_df=pd.DataFrame(data=d)

# Move column
score=final_df["Score"]
final_df.drop("Score", axis=1, inplace=True)
final_df.insert(1, "Score", score)

# Note how high is accuracy score it can be misleading!
final_df

In [ ]:
n_inputs=X_train.shape[1]
undersample_model=Sequential([Dense(n_inputs, input_shape=(n_inputs,), activation="relu"), Dense(32, activation="relu"), Dense(2, activation="softmax")])
undersample_model.summary()

In [ ]:
undersample_model.compile(Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
undersample_model.fit(X_train,y_train,validation_split=0.2,batch_size=25,epochs=20,shuffle=True,verbose=2)
undersample_predictions=undersample_model.predict(original_Xtest, batch_size=200, verbose=0)
undersample_fraud_predictions=np.argmax(undersample_predictions, axis=1)

In [ ]:
# Create a confusion matrix

def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm=cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")

    print(cm)

    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title, fontsize=14)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt=".2f" if normalize else "d"
    thresh=cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")


undersample_cm=confusion_matrix(original_ytest, undersample_fraud_predictions)
actual_cm=confusion_matrix(original_ytest, original_ytest)
labels=["No Fraud", "Fraud"]

In [ ]:
fig=plt.figure(figsize=(16, 8))

fig.add_subplot(221)
plot_confusion_matrix(undersample_cm, labels, title="Random UnderSample \n Confusion Matrix", cmap=plt.cm.Reds)

fig.add_subplot(222)
plot_confusion_matrix(actual_cm, labels, title="Confusion Matrix \n (with 100% accuracy)", cmap=plt.cm.Greens)

In [ ]:
n_inputs=Xsm_train.shape[1]

oversample_model=Sequential([Dense(n_inputs, input_shape=(n_inputs,), activation="relu"), Dense(32, activation="relu"), Dense(2, activation="softmax")])
oversample_model.compile( Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
oversample_model.fit( Xsm_train, ysm_train, validation_split=0.2, batch_size=300, epochs=20, shuffle=True, verbose=2)
oversample_predictions=oversample_model.predict( original_Xtest, batch_size=200, verbose=0)
oversample_fraud_predictions=np.argmax(oversample_predictions, axis=1)

oversample_smote=confusion_matrix(original_ytest, oversample_fraud_predictions)
actual_cm=confusion_matrix(original_ytest, original_ytest)
labels=["No Fraud", "Fraud"]

In [ ]:
fig=plt.figure(figsize=(16, 8))

fig.add_subplot(221)
plot_confusion_matrix(oversample_smote, labels, title="OverSample (SMOTE) \n Confusion Matrix", cmap=plt.cm.Oranges)

fig.add_subplot(222)
plot_confusion_matrix(actual_cm, labels, title="Confusion Matrix \n (with 100% accuracy)", cmap=plt.cm.Greens)